In [1]:
import tensor_comprehensions as tc
import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt

import os
import sys
import logging
import time

ModuleNotFoundError: No module named 'tensor_comprehensions'

In [ ]:
# as recommended by the authors for better performance
tune_settings = {
    "threads": 32,
    "generations": 8,
    "pop_size": 25,
    "number_elites": 3
}

In [2]:
conv_lang = tc.database['convolution_strided']['lang']

NameError: name 'tc' is not defined

In [ ]:
# try some different sizes

I = torch.randn(1, 3, 500, 200).cuda()
W1 = torch.randn(6, 3, 25, 3).cuda()
B = torch.randn(6).cuda()

In [ ]:
conv = tc.define(conv_lang.format(sh=5, sw=1), name='conv')
conv.autotune(I, W1, B,
              cache='cache/tall_conv.tc',
              **tune_settings,
              options=tc.Options('conv'))

In [ ]:
# run twice to prep cuda
conv(I, W1, B, cache='cache/tall_conv.tc')
torch.cuda().synchronize()
conv(I, W1, B, cache='cache/tall_conv.tc')
torch.cuda().synchronize()

In [ ]:
# test performance using TC
output = torch.zeros(1, 6, 500, 200)
torch.cuda.synchronize()

timings = np.zeros(500)

for i in range(500):
    start = time.perf_counter()
    conv(I, W1, B,
         cache='cache/tall_conv.tc', 
         outputs=output)
    torch.cuda.synchronize()
    timings[i] = time.perf_counter() - start

total_elapsed = np.sum(timings)
print('total time: ' + str(total_elapsed))

# get percentile statistics
percentiles = np.percentile(timings, [0, 50, 90])
print(percentiles)


In [ ]:
# test the torch equivalent

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(3, 6, (25, 3), padding=(12, 1)))
    
    def forward(self, x):
        return self.layer(x)

CN = ConvNet()
CN.cuda()

var_input = Variable(torch.randn(1, 3, 500, 200).cuda())
# run a couple times to work out start hiccups
CN(var_input)
torch.cuda.synchronize()
CN(var_input)
torch.cuda.synchronize()


# test performance
timings = np.zeros(500)
output = torch.randn(1, 6, 500, 200).cuda()

for i in range(500):
    start = time.perf_counter()
    output = MN(var_input)
    torch.cuda.synchronize()
    timings[i] = time.perf_counter() - start

total_elapsed = np.sum(timings)
print('total time: ' + str(total_elapsed))

# get percentile statistics
percentiles = np.percentile(timings, [0, 50, 90])
print(percentiles)